In [16]:
import open3d as o3d

In [17]:
import numpy as np
from scipy import linalg
import time
import pickle
import torch
from natsort import natsorted
import os
from glob import glob
from scipy.spatial.transform import Rotation as R
import numpy as np

import numpy as np
# from open3d import JVisualizer

class random_mapping_method():

    '''
    pamameters:
    targetDimen: int, the dimension of the target features
    actiFunc: string, the activation function defined within the class as follows
    scaleRate: float: the scale of the random weights 
    '''
    def __init__(self,targetDimen=100, actiFunc='sin', scaleRate=1):
        self.actiFunc = actiFunc
        self.targetDimen = targetDimen
        self.scaleRate = scaleRate
    
    def feature_mapping(self,dataSet):
        initial_dim = np.size(dataSet, 1)
        self.randomWeights = (np.random.rand(initial_dim, self.targetDimen)*2-1)*self.scaleRate
        self.randomBias = (np.random.rand(1, self.targetDimen)*2-1)*self.scaleRate
        #activation functions, not limited to the followings
        def sigmoid(dataSet):
            return 1.0 / (1 + np.exp(-dataSet))
        def sin(dataSet):
            return np.sin(dataSet)
        def linear(dataSet):
            return dataSet
        def tanh(dataSet):
            return np.tanh(dataSet)
            
        actiFun = {'sig':sigmoid, 'sin':sin, 'linear':linear, 'tanh':tanh}
        
        randomSetTemp = np.dot(dataSet, self.randomWeights) + np.tile(self.randomBias, (len(dataSet), 1))
        randomSet = actiFun[self.actiFunc](randomSetTemp)
        return randomSet

    '''
    Compute least-squares solution of the linear regression model, and other method can also be used.
    parameters:
    X: Training data, array-like of shape (n_samples, n_features). In the context of rmm, it will be the generated randomSet.
    Y: Target values, array-like of shape (n_samples,)
    '''
    def fit(self, X, Y):
        X = np.c_[X, np.ones(len(X))]  # Augment features to yield intercept
        self.coef_, self._residues, self.rank_, self.singular_ = linalg.lstsq(X, Y)

    '''
    Predict the targets using the fitted linear model
    parameter:
    X2pedict: Test samples, array-like of shape (n_samples, n_features). 
              It must be transformed by the same random mapping with the training data.
    '''
    def predict(self, X2pedict):
        X2pedict = np.c_[X2pedict, np.ones(len(X2pedict))]
        Y_Predictd = np.matmul(X2pedict, self.coef_) 
        return Y_Predictd
    
    '''
    Return the coefficient of determination and the mean square error of the prediction.
    parameters:
    X: Test samples, array-like of shape (n_samples, n_features).
    Y: True values for X, array-like of shape (n_samples,).
    '''
    def score(self, X,Y):
        Y_Predictd = self.predict(X)
        Y_mean = np.mean(Y)
        S_tol = np.sum((Y-Y_mean)**2)
        S_reg = np.sum((Y_Predictd-Y)**2)
        R2 = 1 - S_reg/S_tol
        mse = ((Y-Y_Predictd)**2).sum() / len(Y)
        return R2, mse

In [4]:
rellis_root

'/Users/yukiya/ws/dataset/EINRUL'

In [6]:
def load_pkl(path):
    with open(path, 'rb') as f:
        pkl = pickle.load(f)
    return pkl

def transform_pcd(pcd, quaternion, translation):
    # クォータニオンから回転行列を生成
    rotation_matrix = R.from_quat([quaternion['x'], quaternion['y'], quaternion['z'], quaternion['w']]).as_matrix()

    # 変換行列を作成
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = [translation['x'], translation['y'], translation['z']]

    # 点群に変換を適用
    pcd.transform(transform_matrix)

    return pcd

def transform_pcd_from_dict(pcd, trans_dict):
    return transform_pcd(pcd, trans_dict['rotation'], trans_dict['position'])

# rellis(einrul)
rellis_root = '/Users/yukiya/ws/dataset/EINRUL'
sec = os.listdir(rellis_root)
pose_path = os.path.join(rellis_root, sec[0], 'poses.pkl')
pcd_path = natsorted(glob(os.path.join(rellis_root, sec[0], "**/*.pcd"), recursive=True))

# scannet
# pose_path = '../scene/scannet/scene0000_00/final_pose.pkl'
# final_pcd_path = '../scene/scannet/scene0000_00/final_pcd.pcd'

# rellis-3d
# pose_path = '../scene/rellis-3d/0/final_pose.pkl'
# final_pcd_path = '../scene/rellis-3d/0/final_pcd.pcd'

with open(pose_path, 'rb') as f:
    poses = pickle.load(f)

print(f"available {len(pcd_path)} pcd files and {len(poses)} poses.")

available 2059 pcd files and 2059 poses.


In [8]:
loaded_pose = load_pkl(pose_path)
pcd = o3d.io.read_point_cloud(pcd_path[0])

In [13]:
N_SAMPLES = 80000
xyz = np.asarray(pcd.points)
xy = xyz[:, :-1]
z = xyz[:, -1]
rmm = random_mapping_method(targetDimen=500, actiFunc='sin', scaleRate=4)


data_transformed = rmm.feature_mapping(xy)
x_training, y_training, x_test, y_test = data_transformed[:N_SAMPLES,:], z[:N_SAMPLES], data_transformed[N_SAMPLES:,:], z[N_SAMPLES:]

rmm.fit(x_training, y_training)

In [18]:
y_predicted = rmm.predict(np.r_[x_training, x_test])

r2_training, mseTraining = rmm.score(x_training, y_training)
r2_test, mseTest = rmm.score(x_test, y_test)
# key results
print('r2_training:%F, r2_test:%F, mseTraining:%f, mseTest:%f' \
    % (r2_training, r2_test, mseTraining, mseTest))

r2_training:0.826163, r2_test:-0.936472, mseTraining:0.100696, mseTest:0.943818


In [25]:
src_pcd = o3d.geometry.PointCloud()
src_pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
